In [1]:
%load_ext autoreload
# %reload_ext autoreload

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from pycaret.regression import *
from pathlib import Path

In [3]:
# from pycaret.utils import version
# version()

In [4]:
%autoreload
import sys
sys.path.append('../')
from pipeline import Pipin
pipin = Pipin()

Current working directory: /Users/matsalexander/Desktop/Forest Gump/mats/pycret


In [5]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [84]:
# can you move the pv_measurment to the end of the df?
df1 = df1_0.copy()
df2 = df2_0.copy()
df3 = df3_0.copy()
categorical_features = pipin.get_categorical_features(df1)
ignore_features = pipin.get_irrelevant_features()


In [85]:
# add weights
# if is 
df1['weight'] = df1['is_estimated:idx'] * 2 + 1
df2['weight'] = df2['is_estimated:idx'] * 2 + 1
df3['weight'] = df3['is_estimated:idx'] * 2 + 1

In [86]:
# ignore_features.append('weight')
# s = setup(df_transformed, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False)
s1 = setup(df1, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
s2 = setup(df2, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
s3 = setup(df3, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, train_size=0.9, fold_shuffle=True)
# s = setup(df_transformed, target="pv_measurement", normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False, imputation_type="iterative", feature_selection="classic", feature_selection_estimator="et", remove_multicollinearity=True, remove_outliers = True, low_variance_threshold = 0.1,)
# s = setup(df_transformed, target="pv_measurement", imputation_type="iterative", feature_selection="classic", feature_selection_estimator="et", remove_multicollinearity=True, remove_outliers = True, low_variance_threshold = 0.1,  normalize=False, categorical_features=categorical_features, experiment_name="pv_power_mw", ignore_features=ignore_features, index=False,)

,Description,Value
0,Session id,5364
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(21282, 53)"
4,Transformed data shape,"(21282, 56)"
5,Transformed train set shape,"(19153, 56)"
6,Transformed test set shape,"(2129, 56)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,44


,Description,Value
0,Session id,793
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(14757, 53)"
4,Transformed data shape,"(14757, 56)"
5,Transformed train set shape,"(13281, 56)"
6,Transformed test set shape,"(1476, 56)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,44


,Description,Value
0,Session id,2413
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(11993, 53)"
4,Transformed data shape,"(11993, 56)"
5,Transformed train set shape,"(10793, 56)"
6,Transformed test set shape,"(1200, 56)"
7,Ignore features,3
8,Ordinal features,3
9,Numeric features,44


In [87]:
# now pass sample weights to lr and fit again
import numpy as np
# weights = np.arange(s1.get_config('X_train').shape[0])
et_fit = s1.create_model('lr')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,377.5268,331555.6318,575.8087,0.8031,2.1131,7.6587
1,389.4376,354189.1896,595.1380,0.8024,2.0498,8.0601
2,371.8755,330844.7939,575.1911,0.8128,2.2031,11.7961
3,384.5877,344916.5557,587.2960,0.8138,2.0509,8.0161
4,388.4549,358559.4049,598.7983,0.7994,2.0685,9.1367
5,406.8085,398028.8641,630.8953,0.7880,2.0816,8.2395
6,391.7976,378478.1585,615.2058,0.7797,2.0802,8.7462
7,386.7775,366460.7163,605.3600,0.8025,2.0821,11.3420
8,391.3364,365714.0111,604.7429,0.7980,2.0994,13.7466


In [91]:
s1.train_transformed.shape


(19153, 56)

In [93]:
weights = list(s1.train_transformed["weight"])
print(len(weights))
et_fit.fit(X = s1.train_transformed, y = s1.test_transformed, sample_weight = weights)
# weights

19153


ValueError: Found input variables with inconsistent numbers of samples: [19153, 2129]

In [8]:
preprocessed_data = s1.dataset_transformed
preprocessed_data
print(preprocessed_data.shape)
print (preprocessed_data.columns)
preprocessed_data.head(20)
# preprocessed_data.describe()
# categorical features are normalized. That is not good.

(34085, 53)
Index(['total_cloud_coverp', 'diffuse_radW', 'air_density_2mkgm3',
       'wind_speed_v_10mms', 'pressure_50mhPa', 'clear_sky_radW',
       'fresh_snow_3hcm', 'cloud_base_aglm', 'fresh_snow_1hcm',
       'msl_pressurehPa', 'direct_rad_1hJ', 'precip_5minmm', 'rain_waterkgm2',
       't_1000hPaK', 'relative_humidity_1000hPap', 'dew_point_2mK',
       'sun_elevationd', 'wind_speed_w_1000hPams', 'snow_waterkgm2',
       'sun_azimuthd', 'snow_densitykgm3', 'visibilitym',
       'clear_sky_energy_1hJ', 'wind_speed_10mms', 'fresh_snow_6hcm',
       'effective_cloud_coverp', 'snow_depthcm', 'snow_melt_10minmm',
       'absolute_humidity_2mgm3', 'super_cooled_liquid_waterkgm2',
       'wind_speed_u_10mms', 'ceiling_height_aglm', 'direct_radW',
       'diffuse_rad_1hJ', 'sfc_pressurehPa', 'pressure_100mhPa',
       'fresh_snow_12hcm', 'fresh_snow_24hcm', 'prob_rimep', 'elevationm',
       'dew_or_rimeidx_0.0', 'dew_or_rimeidx_1.0', 'dew_or_rimeidx_-1.0',
       'is_dayidx', 'is_in_sh

,total_cloud_coverp,diffuse_radW,air_density_2mkgm3,wind_speed_v_10mms,pressure_50mhPa,clear_sky_radW,fresh_snow_3hcm,cloud_base_aglm,fresh_snow_1hcm,msl_pressurehPa,...,is_dayidx,is_in_shadowidx,precip_type_5minidx_0.0,precip_type_5minidx_1.0,precip_type_5minidx_2.0,precip_type_5minidx_3.0,precip_type_5minidx_5.0,snow_driftidx_0.0,is_estimatedidx,pv_measurement
0,97.175003,1.275000,1.22550,1.475,991.474976,2.750000,0.0,1318.824951,0.0,998.049988,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,7.040000
1,98.800003,0.000000,1.29100,1.950,1015.250000,0.000000,0.0,776.224976,0.0,1022.150024,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
2,99.550003,0.000000,1.22275,1.200,989.625000,0.000000,0.0,1236.449951,0.0,996.099976,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
3,99.875000,6.475000,1.26325,-0.750,987.674988,22.075001,0.0,464.100006,0.0,994.599976,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,19.360001
4,100.000000,12.474999,1.27725,-0.125,1017.174988,54.949997,0.0,305.450012,0.0,1024.125000,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,84.699997
5,2.000000,0.000000,1.25225,2.800,978.299988,0.000000,0.0,1099.375000,0.0,984.750000,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
6,100.000000,0.000000,1.23200,-2.525,963.050049,0.000000,0.0,501.174988,0.0,969.724976,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
7,27.900000,88.250000,1.23700,-2.500,1017.900024,644.424988,0.0,9411.625000,0.0,1025.025024,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3967.040039
8,93.300003,127.375000,1.26700,0.750,1010.200012,591.724976,0.0,439.649994,0.0,1017.200012,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1651.540039
9,79.650002,196.725006,1.23325,-1.000,1015.775024,816.599976,0.0,2640.500000,0.0,1022.750000,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,392.260010


In [12]:
# top3 = s1.compare_models(sort='MAE', n_select=3, )
lightgbm1 = s1.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,279.3270,243167.7839,493.1205,0.8663,1.2634,2.8107
1,304.2122,280219.8595,529.3580,0.8411,1.2509,2.4315
2,294.6579,253707.7761,503.6941,0.8579,1.2179,1.6720
3,295.4902,275954.7902,525.3140,0.8401,1.2500,1.8468
4,298.0157,262351.2012,512.2023,0.8511,1.2482,3.3090
5,301.9318,270827.9726,520.4113,0.8474,1.2412,3.1229
6,295.8293,267822.0329,517.5152,0.8468,1.2882,3.6767
7,304.4115,280601.5930,529.7184,0.8502,1.2362,3.1035
8,290.9201,254290.6073,504.2724,0.8554,1.2498,3.1168


In [13]:
et1 = s1.create_model('et')


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,277.1985,245668.2174,495.6493,0.8649,0.7318,2.1075
1,302.4665,286208.0395,534.9841,0.8377,0.7885,1.6340
2,295.2173,262099.4854,511.9565,0.8532,0.7110,1.1567
3,284.6426,258825.9442,508.7494,0.8501,0.7054,1.2446
4,294.9863,262585.0201,512.4305,0.8510,0.7049,1.3154
5,305.1092,283777.7755,532.7080,0.8401,0.7249,1.3775
6,295.8715,272049.0236,521.5832,0.8444,0.7313,1.8153
7,298.3707,269014.6334,518.6662,0.8564,0.7561,1.6948
8,287.1692,261524.1994,511.3944,0.8513,0.7300,1.5872


In [21]:
# top3 = s2.compare_models(sort='MAE', n_select=3)
et2 = s2.create_model('et')
# lightgbm = s.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,48.4739,7596.7304,87.1592,0.8806,0.7466,1.7749
1,49.1382,7850.1850,88.6013,0.8851,0.7538,1.3081
2,47.0630,7304.2953,85.4652,0.8872,0.7280,1.1828
3,47.1731,7250.4922,85.1498,0.8834,0.7801,1.4734
4,45.8573,6700.4565,81.8563,0.8908,0.7906,1.8968
5,48.3783,8020.4468,89.5569,0.8783,0.7771,10.1854
6,48.6502,7353.8543,85.7546,0.8934,0.7559,1.5307
7,47.4978,7731.8479,87.9309,0.8832,0.7735,1.4213
8,46.1040,6801.3370,82.4702,0.8864,0.7950,1.8355


In [22]:
# top3 = s3.compare_models(sort='MAE', n_select=3)
et3 = s3.create_model('et')

# lightgbm = s.create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,41.8051,5082.9030,71.2945,0.8851,0.6511,0.6772
1,43.7563,5791.6608,76.1030,0.8769,0.6400,0.8570
2,39.9690,5326.4220,72.9823,0.8721,0.6197,0.6143
3,41.2518,4784.6401,69.1711,0.8844,0.6387,0.6766
4,41.5609,5372.5217,73.2975,0.8842,0.6697,0.8010
5,41.9760,5705.1374,75.5324,0.8806,0.6129,0.6255
6,39.5368,4937.3311,70.2661,0.8899,0.6551,0.7100
7,43.3221,5432.2582,73.7039,0.8779,0.6673,0.6317
8,42.7923,5164.6993,71.8658,0.8825,0.7204,0.7112


In [17]:

# et3 = s3.tune_model(et3, optimize="MAE")

In [18]:
et1 = s1.finalize_model(et1)

In [23]:
et2 = s2.finalize_model(et2)


In [24]:
et3 = s3.finalize_model(et3)

In [16]:
# plot_model(lightgbm, plot="feature_all")
# plot_model(gump, plot="feature_all")

In [30]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

In [40]:
from utils import get_unique_filename
pred1 = predict_model(et1, data=test1)
pred2 = predict_model(et2, data=test2)
pred3 = predict_model(et3, data=test3)
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred)
# final_prediction.to_csv(get_unique_filename('et3x'))
final_prediction


,prediction
id,
0,0.015400
1,0.180400
2,5.253600
3,177.806200
4,528.867896
...,...
2155,74.568200
2156,24.754800
2157,10.172400


In [20]:
# print("done", res_stack.shape)
# pipin.compare_mae(res_stack)
